In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('dark')

import keras
import tensorflow

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline

from model_persistence import get_persistence_dataset, train_test_split, calculate_errors, plot_error
from features_preprocessing import make_shifted_features, transform_to_windows
from create_day_types import get_days_dummies


In [225]:
def split_sequences(sequences, n_steps, idx=0, multivar=False):
    """
    Function modified for use from Deep learning time series forecasting by Jason Brownlee
    """
    
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :]
        seq_y = sequences[end_ix, :]

        X.append(seq_x)
        y.append(seq_y)
        
    X = np.array(X)
    y = np.array(y)
    
    if multivar:
        #unstack the 3rd dimension and select the first element(energy load)
        y = np.dsplit(y,y.shape[-1])[idx]
        y = np.squeeze(y)
    
    return X, y

In [183]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

in_multi_1 = np.array([1,1,1,1])
in_multi_2 = np.array([1,2,3,4])
in_multi_3 = np.array([2,3,4,5])

# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))

print(dataset)
print(" ")
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)

print(X.shape, y.shape)
# summarize the data
for i in range(2):
    print(X[i], y[i])

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
 
(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]


In [9]:
data_comb = pd.read_csv('./data/cleaned_data/energy_weather_2015_2018.csv', parse_dates=True, index_col='time')
data_comb.head(3)

,actual_load,temp,pressure,wind_speed,rain_1h,rain_3h,snow_3h,heatdd,cooldd
time,,,,,,,,,
2015-01-01 00:00:00,25385.0,271.239327,998.190492,2.078209,0.0,0.0,0.0,19.910673,0.0
2015-01-01 01:00:00,24382.0,271.245271,998.134512,2.078209,0.0,0.0,0.0,19.904729,0.0
2015-01-01 02:00:00,22734.0,270.544537,998.507545,2.230083,0.0,0.0,0.0,20.605463,0.0


In [13]:
data_comb.index.min(), data_comb.index.max()

(Timestamp('2015-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [14]:
days_df = get_days_dummies(start='2015-01-01', stop='2018-12-31', frequency='H')
days_df.head(3)

,mon,tue,wed,thur,fri,sat,sun
2015-01-01 00:00:00,0,0,0,1,0,0,0
2015-01-01 01:00:00,0,0,0,1,0,0,0
2015-01-01 02:00:00,0,0,0,1,0,0,0


In [15]:
days_df.index.min(), days_df.index.max()

(Timestamp('2015-01-01 00:00:00'), Timestamp('2018-12-31 00:00:00'))

In [22]:
## apply PCA pipeline to the weather features.

#initate a min max scaler
minmax = MinMaxScaler()

#initate a PCA object to cpature 90% of dimensionality
pca = PCA(n_components=2)

#initate the pipeline for pca
pca_pipe = make_pipeline(pca, minmax)

minmax_pipe = make_pipeline(minmax)

weather_cols = ['temp','pressure','wind_speed','rain_1h','rain_3h','snow_3h','heatdd','cooldd']

#select the weather columns and make sure there are no Nans before transform
data_weather = data_comb[weather_cols].interpolate(method='linear')

#transform the weather data
pca_weather = pca_pipe.fit_transform(data_weather)

#make dataframe with the transformed weather pca
pca_weather_df = pd.DataFrame(pca_weather, index=data_weather.index, columns=['pca1', 'pca2'])

In [23]:
pca_weather_df.head(3)

,pca1,pca2
time,,
2015-01-01 00:00:00,0.000932,0.051902
2015-01-01 01:00:00,0.000932,0.052050
2015-01-01 02:00:00,0.000933,0.034670


In [24]:
pca_weather_df.index.max()

Timestamp('2018-12-31 23:00:00')

In [162]:
all_data = pd.concat([data_comb['actual_load'], pca_weather_df, days_df], axis=1)
all_data.head(3)

,actual_load,pca1,pca2,mon,tue,wed,thur,fri,sat,sun
2015-01-01 00:00:00,25385.0,0.000932,0.051902,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-01-01 01:00:00,24382.0,0.000932,0.052050,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-01-01 02:00:00,22734.0,0.000933,0.034670,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [26]:
all_data.to_csv('./data/lstm/nn_dataset_2015_2018.csv')

In [170]:
data = pd.read_csv('./data/lstm/nn_dataset_2015_2018.csv', parse_dates=True, index_col=0)
data.head(2)

,actual_load,pca1,pca2,mon,tue,wed,thur,fri,sat,sun
2015-01-01 00:00:00,25385.0,0.000932,0.051902,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-01-01 01:00:00,24382.0,0.000932,0.052050,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [330]:
all_data = pd.read_csv('./data/lstm/nn_dataset_2015_2018.csv', parse_dates=True, index_col=0)

data = all_data['2015-01-01':'2015-01-05'].copy()

all_data.head(3)

,actual_load,pca1,pca2,mon,tue,wed,thur,fri,sat,sun
2015-01-01 00:00:00,25385.0,0.000932,0.051902,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-01-01 01:00:00,24382.0,0.000932,0.052050,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-01-01 02:00:00,22734.0,0.000933,0.034670,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [333]:
check = data['actual_load'].values.reshape(-1,1)

c, s = normalize_df(check)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [240]:
data.loc[:,'year'] = data.index.year
data.loc[:,'month'] = data.index.month
data.loc[:,'day'] = data.index.day
data.loc[:,'hours'] = data.index.hour

In [241]:
data.head(3)

,actual_load,pca1,pca2,mon,tue,wed,thur,fri,sat,sun,year,month,day,hours
2015-01-01 00:00:00,25385.0,0.000932,0.051902,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015,1,1,0
2015-01-01 01:00:00,24382.0,0.000932,0.052050,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015,1,1,1
2015-01-01 02:00:00,22734.0,0.000933,0.034670,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015,1,1,2


In [245]:
hours = [] 

for h in data.hours.unique():
    hour = data[data.hours==0].copy()
    hour.drop(['year', 'month', 'day', 'hours'], axis=1, inplace=True)
    hours.append(hour)

In [247]:
hours[1]

,actual_load,pca1,pca2,mon,tue,wed,thur,fri,sat,sun
2015-01-01,25385.0,0.000932,0.051902,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2015-01-02,27070.0,0.000951,0.071115,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2015-01-03,25564.0,0.001038,0.119969,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2015-01-04,25254.0,0.000936,0.124913,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2015-01-05,25069.0,0.001006,0.141094,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [248]:
hours.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
#reshape so each stack is 3D

for hour in hours:
hours_3d = np.reshape(hours.values, (hours.shape[0], hours.shape[1], 1))
    



In [321]:
data[data.hours==0]

,actual_load,pca1,pca2,mon,tue,wed,thur,fri,sat,sun,year,month,day,hours
2015-01-01,25385.0,0.000932,0.051902,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015,1,1,0
2015-01-02,27070.0,0.000951,0.071115,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2015,1,2,0
2015-01-03,25564.0,0.001038,0.119969,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2015,1,3,0
2015-01-04,25254.0,0.000936,0.124913,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2015,1,4,0
2015-01-05,25069.0,0.001006,0.141094,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2015,1,5,0


In [316]:
hours_tup = [] 

for h in data.hours.unique():
    hour = data[data.hours==h].copy()
    hour.drop(['year', 'month', 'day', 'hours'], axis=1, inplace=True)
    hour = np.reshape(hour.values, (hour.shape[0], hour.shape[1], 1))
    
    hours_tup.append(hour)

In [320]:
hours_tup[0]

array([[[2.53850000e+04],
        [9.31802143e-04],
        [5.19024480e-02],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [1.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[2.70700000e+04],
        [9.51295989e-04],
        [7.11146036e-02],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [1.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00]],

       [[2.55640000e+04],
        [1.03769440e-03],
        [1.19968967e-01],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [1.00000000e+00],
        [0.00000000e+00]],

       [[2.52540000e+04],
        [9.36364229e-04],
        [1.24912660e-01],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
        [0.00000000e+00],
      

In [255]:
tup = tuple(hours_tup)

hours_stacked = np.dstack(tup)

In [253]:
#5 days, 10 features, 1 hour
hours[0].shape

(5, 10, 1)

In [256]:
#5days, 10 features, 24 hours
hours_stacked.shape

(5, 10, 24)

In [257]:
flat_hours = hours_stacked.reshape(-1, hours_stacked.shape[-1])

In [258]:
flat_hours.shape

(50, 24)

In [265]:
flat_hours[:10,0]

array([2.53850000e+04, 9.31802143e-04, 5.19024480e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])

In [266]:
#make samples from hours stacked
X, y = split_sequences(hours_stacked, 2, idx=0, multivar=False)

In [270]:
#3 samples, 2 lags, 10 features, 24 hours
X.shape

(3, 2, 10, 24)

In [272]:
new = []

for j in range(len(X)):
    n = X[j].reshape(-1, hours_stacked.shape[-1])
    new.append(n)
new = np.array(new)

#3 samples, 20 lags*features, 24 hours
new.shape

(3, 20, 24)

In [353]:
X, y, scaler_loads = get_lstm_multivariable_data_3d(start='2015-01-01', stop='2015-01-05', n_lags=2)

In [354]:
X.shape

(3, 20, 24)

In [355]:
y.shape

(3, 24)

In [356]:
y[0]

array([0.41117489, 0.26731091, 0.16631548, 0.10106808, 0.09009664,
       0.10252125, 0.12853302, 0.16144736, 0.2071496 , 0.34505558,
       0.47584102, 0.5125336 , 0.49487757, 0.50112621, 0.47308   ,
       0.39649786, 0.35348398, 0.33393882, 0.36961418, 0.58555547,
       0.71125481, 0.72433336, 0.65930393, 0.50555838])

In [ ]:
n, scaler = normalize_df() 
n.head(3)

25564.0
25254.0
25069.0


In [352]:
#work with a subset
all_data = all_data['2015-01-01': '2015-01-05']

def get_lstm_multivariable_data_3d(start='2015-01-01', stop='2015-01-05', n_lags=2):

    #load in the prepared dataset
    all_data = pd.read_csv('./data/lstm/nn_dataset_2015_2018.csv', parse_dates=True, index_col=0)

    #select data time slice
    data = all_data[start: stop].copy()
    
    #reshape the energy load columns to prepare for minmax scaling
    energy = data['actual_load'].values.reshape(-1,1)
    
    #minmax scale the energy column
    scaler = MinMaxScaler().fit(energy)
    data_normd = scaler.transform(energy)
    
    #reset the energy column to the actual loads
    data['actual_load'] = data_normd.copy()
    
    #create single columns with time features
    data.loc[:,'year'] = data.index.year
    data.loc[:,'month'] = data.index.month
    data.loc[:,'day'] = data.index.day
    data.loc[:,'hours'] = data.index.hour
    
    hours_tup = [] 

    #for each unique hour isolate the features and dates
    for h in data.hours.unique():
        
        #boolean mask for each hour of the day
        hour = data[data.hours==h].copy()
        #drop the unneeded columns
        hour.drop(['year', 'month', 'day', 'hours'], axis=1, inplace=True)
        #reshape 2D into a 3D matrix for stacking
        hour = np.reshape(hour.values, (hour.shape[0], hour.shape[1], 1))
        #append each 3d slice into list
        hours_tup.append(hour)
    
    
    tup = tuple(hours_tup)
    
    #stack all the 3D arraysinto 1 single 3D array
    hours_stacked = np.dstack(tup)
    
    #make samples from hours stacked. result is 4D and 2D
    X_4d, y = split_sequences(hours_stacked, n_lags, idx=0, multivar=True)
    
    
    X = []

    #flatten the 2nd and 3rd dimensions together to have a final array of samples, lags & features, hours
    for j in range(len(X_4d)):
        #reshape the inner dimensions
        n = X_4d[j].reshape(-1, hours_stacked.shape[-1])
        X.append(n)
    
    X = np.array(X)
    
    
    return X, y, scaler

In [292]:
def split_sequences(sequences, n_steps, idx=0, multivar=False):
    """
    Function modified for use from Deep learning time series forecasting by Jason Brownlee
    """
    
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequences[i:end_ix, :]
        seq_y = sequences[end_ix, :]

        X.append(seq_x)
        y.append(seq_y)
        
    X = np.array(X)
    y = np.array(y)
    
    if multivar:
        #retun only the feature target in the indx column 
        y = y[:,idx]
    
    return X, y

In [224]:
#work with a subset
all_data = all_data['2015-01-01': '2015-01-05']

def get_lstm_multivariable_data(start='2015-01-01', stop='2015-01-05', n_lags=2):

    #load in the prepared dataset
    all_data = pd.read_csv('./data/lstm/nn_dataset_2015_2018.csv', parse_dates=True, index_col=0)

    #select data time slice
    data = all_data[start: stop].copy()
    
    #list to store the windows of 2D feature data
    windowed_data = []

    #for each 2d feautre window 
    for col in data.columns:
        #transform each column to row of dates, with columns as the hourly slices
        window = transform_to_windows(data, load_type=col)

        #if the column is actual load, we need to scale it, otherwise all columns are already scaled
        if col == 'actual_load':
            window, scaler = normalize_df(window)

        #reshape each 2D feature window into 3D so it can be stacked
        window = np.reshape(window.values, (window.shape[0], window.shape[1], 1))
        
        #append to list
        windowed_data.append(window)
        
    #convert list of 2D feature windows to a numpy array
    windowed_data = np.array(windowed_data)

    tup = []

    #make a list of each 2D dataslice to stack
    for i in range(len(windowed_data)):
        tup.append(windowed_data[i])

    #tun into a tuple
    tup = tuple(tup)

    #stack up the 2d feature windows to a 3D array
    windows_stacked = np.dstack(tup)

    #split the 3D data into 4D slices of X and y. multivar True to select only the index value of the y array.
    #indx is the location of the y target values.
    X, y = split_sequences(windows_stacked, n_lags, idx=0, multivar=True)
    
    return X, y, scaler


In [226]:
X, y, scaler = get_lstm_multivariable_data(start='2015-01-01', stop='2015-01-05')

In [227]:
X.shape, y.shape

((3, 2, 24, 10), (3, 24))

In [223]:
y

array([[[0.24737631],
        [0.12556634],
        [0.1680261 ],
        [0.08394698],
        [0.4808952 ],
        [0.61615721],
        [0.41341615],
        [0.31608943],
        [0.32376551],
        [0.43502274],
        [0.50437912],
        [0.47224913],
        [0.40036259],
        [0.46074933],
        [0.52126758],
        [0.45238998],
        [0.39264251],
        [0.39773732],
        [0.43370127],
        [0.59643109],
        [0.65680246],
        [0.52951389],
        [0.51071682],
        [0.51691314]],

       [[0.09245377],
        [0.14886731],
        [0.5040783 ],
        [0.74300442],
        [0.90884279],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.9152686 ],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.79488613],
        [0.54138522],
        [0.55207533],
        

In [201]:
y

array([[[2.47376312e-01, 1.03769440e-03, 1.19968967e-01, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [1.25566343e-01, 9.54558913e-04, 1.00556206e-01, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [1.68026101e-01, 9.53887226e-04, 9.62076719e-02, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [8.39469809e-02, 9.53593813e-04, 8.00140870e-02, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [4.80895197e-01, 9.53593815e-04, 7.99217486e-02, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         1.00000000e+00, 0.00000000e+00],
        [6.16157205e-01, 9.52628302e-04, 7.99322091e-02, 0.0

In [209]:
y.shape

(3, 24, 10)

In [215]:
np.dsplit(y,y.shape[-1])[0]

array([[[0.24737631],
        [0.12556634],
        [0.1680261 ],
        [0.08394698],
        [0.4808952 ],
        [0.61615721],
        [0.41341615],
        [0.31608943],
        [0.32376551],
        [0.43502274],
        [0.50437912],
        [0.47224913],
        [0.40036259],
        [0.46074933],
        [0.52126758],
        [0.45238998],
        [0.39264251],
        [0.39773732],
        [0.43370127],
        [0.59643109],
        [0.65680246],
        [0.52951389],
        [0.51071682],
        [0.51691314]],

       [[0.09245377],
        [0.14886731],
        [0.5040783 ],
        [0.74300442],
        [0.90884279],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.9152686 ],
        [1.        ],
        [1.        ],
        [1.        ],
        [0.79488613],
        [0.54138522],
        [0.55207533],
        

In [167]:
X.shape, y.shape

((3, 2, 24, 2), (3, 24, 1))

In [168]:
y[0]

array([[0.24737631],
       [0.12556634],
       [0.1680261 ],
       [0.08394698],
       [0.4808952 ],
       [0.61615721],
       [0.41341615],
       [0.31608943],
       [0.32376551],
       [0.43502274],
       [0.50437912],
       [0.47224913],
       [0.40036259],
       [0.46074933],
       [0.52126758],
       [0.45238998],
       [0.39264251],
       [0.39773732],
       [0.43370127],
       [0.59643109],
       [0.65680246],
       [0.52951389],
       [0.51071682],
       [0.51691314]])

In [165]:
windowed_data = []

for col in ['actual_load', 'pca1']:
    window = transform_to_windows(all_data, load_type=col)
    
    if col == 'actual_load':
        window, scaler = normalize_df(window)
        
    window = np.reshape(window.values, (window.shape[0], window.shape[1], 1))
    
    windowed_data.append(window)
    
windowed_data = np.array(windowed_data)



In [164]:
load = transform_to_windows(all_data, load_type='actual_load')

load_scaled, scaler = normalize_df(load)

load_scaled.head(3)

hours,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0.368894,0.425675,0.394568,0.307336,0.235866,0.204723,0.156752,0.110786,0.080487,0.083461,...,0.112874,0.134571,0.131394,0.137901,0.287402,0.421270,0.349116,0.313648,0.380439,0.359445
2015-01-02,0.538958,0.482880,0.446714,0.449079,0.443012,0.442065,0.356664,0.275504,0.267323,0.339357,...,0.380364,0.373075,0.351187,0.330388,0.425037,0.598083,0.579727,0.586418,0.640055,0.637494
2015-01-03,0.386960,0.343126,0.335904,0.308466,0.335482,0.359692,0.303619,0.219644,0.218450,0.281255,...,0.214976,0.232977,0.217805,0.186416,0.210320,0.418761,0.385996,0.333822,0.376462,0.365662


In [350]:
def normalize_df(data):
    
    #normalize the dataset for working with the lstm nn
    scaler = MinMaxScaler().fit(data.values)
    data_normd = scaler.transform(data.values)

    data = pd.DataFrame(data_normd, index=data.index, columns=data.columns)
    
    return data, scaler

In [339]:
normalize_df(data)

yes


(                     actual_load      pca1      pca2  mon  tue  wed  thur  \
 2015-01-01 00:00:00     0.398169  0.145574  0.062729  0.0  0.0  0.0   1.0   
 2015-01-01 01:00:00     0.325292  0.144370  0.063069  0.0  0.0  0.0   1.0   
 2015-01-01 02:00:00     0.205551  0.152397  0.023148  0.0  0.0  0.0   1.0   
 2015-01-01 03:00:00     0.100341  0.151192  0.022993  0.0  0.0  0.0   1.0   
 2015-01-01 04:00:00     0.026084  0.151192  0.023895  0.0  0.0  0.0   1.0   
 ...                          ...       ...       ...  ...  ...  ...   ...   
 2015-01-05 19:00:00     0.155707  0.202620  0.465958  1.0  0.0  0.0   0.0   
 2015-01-05 20:00:00     0.214561  0.017899  0.323039  1.0  0.0  0.0   0.0   
 2015-01-05 21:00:00     0.414081  0.017899  0.323621  1.0  0.0  0.0   0.0   
 2015-01-05 22:00:00     0.397878  0.013832  0.327638  1.0  0.0  0.0   0.0   
 2015-01-05 23:00:00     0.284604  0.000000  0.279606  1.0  0.0  0.0   0.0   
 
                      fri  sat  sun  
 2015-01-01 00:00:00  0.

In [156]:
np.dstack((windowed_data[0], windowed_data[1])).shape

(5, 24, 2)

In [155]:
windows_stacked.shape

(5, 24, 2)

In [61]:
first_try = np.dstack((windowed_data[0], windowed_data[1]))

In [62]:
#(days, hour slices, features)
first_try.shape

(5, 24, 2)

In [63]:
first_try

array([[[2.53850000e+04, 9.31802143e-04],
        [2.43820000e+04, 9.31508398e-04],
        [2.27340000e+04, 9.33466194e-04],
        [2.12860000e+04, 9.33172453e-04],
        [2.02640000e+04, 9.33172445e-04],
        [1.99050000e+04, 9.38210531e-04],
        [2.00100000e+04, 9.38210526e-04],
        [2.03770000e+04, 9.38504227e-04],
        [2.00940000e+04, 9.46746615e-04],
        [2.06370000e+04, 9.46746585e-04],
        [2.22500000e+04, 9.46746437e-04],
        [2.35470000e+04, 9.48000747e-04],
        [2.41330000e+04, 9.48000638e-04],
        [2.47130000e+04, 9.48000681e-04],
        [2.46720000e+04, 1.11685858e-03],
        [2.35280000e+04, 1.11881738e-03],
        [2.31180000e+04, 1.12030350e-03],
        [2.36060000e+04, 1.12377364e-03],
        [2.64470000e+04, 1.12706483e-03],
        [2.80200000e+04, 1.01920696e-03],
        [2.90140000e+04, 1.03752251e-03],
        [2.95710000e+04, 9.48005308e-04],
        [2.90310000e+04, 9.48005298e-04],
        [2.67980000e+04, 9.5129598

In [150]:
X, y = modified_split_sequences(first_try, 2, idx=0)

(3, 24, 2)


In [106]:
#(samples, lags, hourly slices, features)
X.shape

(3, 2, 24, 2)

In [91]:
X[0][0]

array([[2.53850000e+04, 9.31802143e-04],
       [2.43820000e+04, 9.31508398e-04],
       [2.27340000e+04, 9.33466194e-04],
       [2.12860000e+04, 9.33172453e-04],
       [2.02640000e+04, 9.33172445e-04],
       [1.99050000e+04, 9.38210531e-04],
       [2.00100000e+04, 9.38210526e-04],
       [2.03770000e+04, 9.38504227e-04],
       [2.00940000e+04, 9.46746615e-04],
       [2.06370000e+04, 9.46746585e-04],
       [2.22500000e+04, 9.46746437e-04],
       [2.35470000e+04, 9.48000747e-04],
       [2.41330000e+04, 9.48000638e-04],
       [2.47130000e+04, 9.48000681e-04],
       [2.46720000e+04, 1.11685858e-03],
       [2.35280000e+04, 1.11881738e-03],
       [2.31180000e+04, 1.12030350e-03],
       [2.36060000e+04, 1.12377364e-03],
       [2.64470000e+04, 1.12706483e-03],
       [2.80200000e+04, 1.01920696e-03],
       [2.90140000e+04, 1.03752251e-03],
       [2.95710000e+04, 9.48005308e-04],
       [2.90310000e+04, 9.48005298e-04],
       [2.67980000e+04, 9.51295985e-04]])

In [151]:
y.shape

(3, 24, 1)

In [152]:
y[0]

array([[25564.],
       [23584.],
       [22194.],
       [21296.],
       [21145.],
       [21316.],
       [21674.],
       [22127.],
       [22756.],
       [24654.],
       [26454.],
       [26959.],
       [26716.],
       [26802.],
       [26416.],
       [25362.],
       [24770.],
       [24501.],
       [24992.],
       [27964.],
       [29694.],
       [29874.],
       [28979.],
       [26863.]])

In [143]:
np.dsplit(y,2)

[array([[[25564.],
         [23584.],
         [22194.],
         [21296.],
         [21145.],
         [21316.],
         [21674.],
         [22127.],
         [22756.],
         [24654.],
         [26454.],
         [26959.],
         [26716.],
         [26802.],
         [26416.],
         [25362.],
         [24770.],
         [24501.],
         [24992.],
         [27964.],
         [29694.],
         [29874.],
         [28979.],
         [26863.]],
 
        [[25254.],
         [23620.],
         [22606.],
         [22191.],
         [21929.],
         [22195.],
         [24035.],
         [26807.],
         [28316.],
         [29871.],
         [30585.],
         [30772.],
         [30685.],
         [30429.],
         [28741.],
         [28249.],
         [28105.],
         [27908.],
         [27577.],
         [27418.],
         [28604.],
         [30346.],
         [28190.],
         [26270.]],
 
        [[25069.],
         [23390.],
         [21988.],
         [21182.],
      